## The main objective of the project will be to find ideal spots in the city where fast food retail chains can be put up, aiming at the above demographic, thereby helping the owners of the outlets to extract maximum profits out of them.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans


print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


In [2]:
#Get the data from the URL
data = requests.get('https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Mumbai').text

### Now we will use beautiful Soup package to get data

In [3]:
#import and use beautifulsoup
from bs4 import BeautifulSoup
soup = BeautifulSoup(data,'html.parser')

#Create lists
area = []
locationinarea = []
lattitude = []
longitude = []

#Append the data
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        area.append(cells[0].text.rstrip('\n'))
        locationinarea.append(cells[1].text.rstrip('\n'))
        lattitude.append(cells[2].text.rstrip('\n'))
        longitude.append(cells[3].text.rstrip('\n'))
      


Let's create a dataframe from the imported data

In [4]:
#create a dataframe
Mumbai_df = pd.DataFrame({"Area": area,
                           "Location in Mumbai": locationinarea,
                           "Lattitude": lattitude,
                           "Longitude": longitude})
v1=lambda x:float(x[0:-1])
Mumbai_df.Longitude = Mumbai_df.Longitude.apply(v1)
Mumbai_df.Lattitude = Mumbai_df.Lattitude.apply(v1)
Mumbai_df


,Area,Location in Mumbai,Lattitude,Longitude
0,Amboli,"Andheri,Western Suburbs",19.12900,72.84300
1,"Chakala, Andheri",Western Suburbs,19.11138,72.86083
2,D.N. Nagar,"Andheri,Western Suburbs",19.12408,72.83137
3,Four Bungalows,"Andheri,Western Suburbs",19.12471,72.82720
4,Lokhandwala,"Andheri,Western Suburbs",19.13081,72.82920
5,Marol,"Andheri,Western Suburbs",19.11921,72.88274
6,Sahar,"Andheri,Western Suburbs",19.09888,72.86722
7,Seven Bungalows,"Andheri,Western Suburbs",19.12905,72.81701
8,Versova,"Andheri,Western Suburbs",19.10000,72.80000
9,Mira Road,"Mira-Bhayandar,Western Suburbs",19.28416,72.87111


In [5]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')


Solving environment: ...working... done

# All requested packages already installed.

Folium installed


In [6]:
lat = 19.0760
lng = 72.8777
print('The geograpical coordinate of Mumbai are {}, {}.'.format(lat, lng))



The geograpical coordinate of Mumbai are 19.076, 72.8777.


In [7]:
# create map of Mumbai using latitude and longitude values
map_mumbai = folium.Map(location=[lat, lng], zoom_start=12)


# add markers to map
for lat, lng,area, locat in zip(Mumbai_df['Lattitude'], Mumbai_df['Longitude'],Mumbai_df['Area'],Mumbai_df['Location in Mumbai']):
    label = '{},{}'.format(area,locat)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.2).add_to(map_mumbai)    
    
map_mumbai

# Use the Foursquare API to explore the neighborhoods

In [8]:
# define Foursquare Credentials and Version
CLIENT_ID = 'JOB0Y2JORU20NTASROWV5K1KY11MPMWY3O24VHT4ZKJ0CSAH' # your Foursquare ID
CLIENT_SECRET = 'I3MGK4JNAVMNVRPBSN42BHB4IWZHSFNYKKUFMD0MLKZXTOVZ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: JOB0Y2JORU20NTASROWV5K1KY11MPMWY3O24VHT4ZKJ0CSAH
CLIENT_SECRET:I3MGK4JNAVMNVRPBSN42BHB4IWZHSFNYKKUFMD0MLKZXTOVZ


In [9]:
radius = 100
LIMIT = 50

venues = []

for lat, long, area, location in zip(Mumbai_df['Lattitude'], Mumbai_df['Longitude'], Mumbai_df['Area'], Mumbai_df['Location in Mumbai'],):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            area, 
            location,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [10]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Area', 'Location in Mumbai', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df

(97, 8)


,Area,Location in Mumbai,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Amboli,"Andheri,Western Suburbs",19.12900,72.84300,Joe & Helen D'mello Ground,19.129238,72.842795,Playground
1,"Chakala, Andheri",Western Suburbs,19.11138,72.86083,The Mirador Mumbai,19.111462,72.860667,Asian Restaurant
2,Seven Bungalows,"Andheri,Western Suburbs",19.12905,72.81701,Tanjore Tiffin Room,19.128438,72.817150,South Indian Restaurant
3,Seven Bungalows,"Andheri,Western Suburbs",19.12905,72.81701,Tirupati Dosa Center,19.128269,72.816854,Indian Restaurant
4,Seven Bungalows,"Andheri,Western Suburbs",19.12905,72.81701,WTF Versova,19.128465,72.817373,Pub
5,Seven Bungalows,"Andheri,Western Suburbs",19.12905,72.81701,Malwani Kalwan,19.128793,72.816619,Seafood Restaurant
6,Seven Bungalows,"Andheri,Western Suburbs",19.12905,72.81701,The Treesome Café,19.128635,72.817322,Bistro
7,Seven Bungalows,"Andheri,Western Suburbs",19.12905,72.81701,Baskin-Robbins,19.128966,72.816204,Ice Cream Shop
8,Seven Bungalows,"Andheri,Western Suburbs",19.12905,72.81701,Costa Coffee,19.129040,72.817541,Café
9,Mira Road,"Mira-Bhayandar,Western Suburbs",19.28416,72.87111,borivali biryani centre,19.283890,72.871126,Indian Restaurant


###### Let's check how many venues were returned for each PostalCode

In [11]:
venues_df.groupby(["Area", "Location in Mumbai",]).count()

,,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Area,Location in Mumbai,,,,,,
Amboli,"Andheri,Western Suburbs",1,1,1,1,1,1
Bandstand Promenade,"Bandra,Western Suburbs",4,4,4,4,4,4
C.G.S. colony,"Antop Hill,South Mumbai",1,1,1,1,1,1
Cavel,South Mumbai,1,1,1,1,1,1
"Chakala, Andheri",Western Suburbs,1,1,1,1,1,1
Chembur,Harbour Suburbs,1,1,1,1,1,1
Cotton Green,South Mumbai,3,3,3,3,3,3
Cumbala Hill,South Mumbai,1,1,1,1,1,1
Dahisa,Western Suburbs,2,2,2,2,2,2


#### Let's analyze each area now

In [12]:
# one hot encoding
Mum_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")
Mum_onehot
# add postal, borough and neighborhood column back to dataframe
Mum_onehot['Area'] = venues_df['Area'] 
Mum_onehot['Location in Mumbai'] = venues_df['Location in Mumbai'] 


# move postal, borough and neighborhood column to the first column
fixed_columns = list(Mum_onehot.columns[-2:]) + list(Mum_onehot.columns[:-2])
Mum_onehot = Mum_onehot[fixed_columns]

print(Mum_onehot.shape)
Mum_onehot.head(97)

(97, 47)


,Area,Location in Mumbai,Aquarium,Arcade,Asian Restaurant,Bakery,Bar,Bistro,Burger Joint,Café,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Department Store,Dessert Shop,Fast Food Restaurant,Food Truck,Hookah Bar,Hotel,Ice Cream Shop,Indian Restaurant,Lounge,Market,Men's Store,Middle Eastern Restaurant,Music Venue,Neighborhood,Performing Arts Venue,Platform,Playground,Plaza,Pool,Pub,Restaurant,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shopping Mall,Smoke Shop,Snack Place,South Indian Restaurant,Spanish Restaurant,Train Station,Video Store
0,Amboli,"Andheri,Western Suburbs",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Chakala, Andheri",Western Suburbs,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Seven Bungalows,"Andheri,Western Suburbs",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,Seven Bungalows,"Andheri,Western Suburbs",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Seven Bungalows,"Andheri,Western Suburbs",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
5,Seven Bungalows,"Andheri,Western Suburbs",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
6,Seven Bungalows,"Andheri,Western Suburbs",0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,Seven Bungalows,"Andheri,Western Suburbs",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,Seven Bungalows,"Andheri,Western Suburbs",0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,Mira Road,"Mira-Bhayandar,Western Suburbs",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [57]:
num_top_venues = 3

indicators = ['st', 'nd', ]

# create columns according to number of top venues
area = ['Area', 'Location in Mumbai', ]
freq = []
for ind in np.arange(num_top_venues):
    try:
        freq.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freq.append('{}th Most Common Venue'.format(ind+1))
columns = area+freq

# create a new dataframe
Mum_venues_sorted = pd.DataFrame(columns=columns)
Mum_venues_sorted['Area'] = Mum_onehot['Area']
Mum_venues_sorted['Location in Mumbai'] = Mum_onehot['Location in Mumbai']


for ind in np.arange(Mum_onehot.shape[0]):
    row = Mum_onehot.iloc[ind, :].iloc[2:]
    row_sorted = row.sort_values(ascending=False)
    Mum_venues_sorted.iloc[ind, 2:] = row_sorted.index.values[0:num_top_venues]

Mum_venues_sorted.sort_values(freq, inplace=True)
print(Mum_venues_sorted.shape)
Mum_venues_sorted

(97, 5)


,Area,Location in Mumbai,1st Most Common Venue,2nd Most Common Venue,3th Most Common Venue
23,Gorai,"Borivali (West),Western Suburbs",Aquarium,Indian Restaurant,Hotel
35,Pali Naka,"Khar,Western Suburbs",Arcade,Video Store,Cocktail Bar
56,Hiranandani Gardens,"Powai,Eastern Suburbs",Arcade,Video Store,Cocktail Bar
1,"Chakala, Andheri",Western Suburbs,Asian Restaurant,Video Store,Cocktail Bar
65,Hiranandani Gardens,"Powai,Eastern Suburbs",Asian Restaurant,Video Store,Cocktail Bar
21,I.C. Colony,"Borivali (West),Western Suburbs",Bakery,Video Store,Cocktail Bar
52,Hiranandani Gardens,"Powai,Eastern Suburbs",Bakery,Video Store,Cocktail Bar
74,Dhobitalao,South Mumbai,Bakery,Video Store,Cocktail Bar
15,Kherwadi,"Bandra,Western Suburbs",Bar,Video Store,Cocktail Bar
20,I.C. Colony,"Borivali (West),Western Suburbs",Bar,Video Store,Cocktail Bar


###### Let's remove all the Area where food places are the 1st most common venue

In [113]:
Mum_venues_sorted_df_new = Mum_venues_sorted.groupby(['1st Most Common Venue']).size().reset_index(name='count')

Mum_venues_sorted_df.rename(columns={'1st Most Common Venue': 'col'}, inplace=True)

searchfor = ['Restaurant', 'Bar','Burger Joint','Pizza','Snack','Shop','Pub','Food',"Cafe","Bakery",'Café','Lounge','Sandwich']
Mum_venues_sorted_df_new = Mum_venues_sorted_df[~Mum_venues_sorted_df.col .str.contains('|'.join(searchfor))]
Mum_onehot_new= pd.merge(Mum_venues_sorted_df_new, Mum_onehot, left_index=True, right_index=True)
Mum_onehot_new

,col,count,Area,Location in Mumbai,Aquarium,Arcade,Asian Restaurant,Bakery,Bar,Bistro,Burger Joint,Café,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Department Store,Dessert Shop,Fast Food Restaurant,Food Truck,Hookah Bar,Hotel,Ice Cream Shop,Indian Restaurant,Lounge,Market,Men's Store,Middle Eastern Restaurant,Music Venue,Neighborhood,Performing Arts Venue,Platform,Playground,Plaza,Pool,Pub,Restaurant,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shopping Mall,Smoke Shop,Snack Place,South Indian Restaurant,Spanish Restaurant,Train Station,Video Store
0,Aquarium,1,Amboli,"Andheri,Western Suburbs",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Arcade,2,"Chakala, Andheri",Western Suburbs,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,Bistro,1,Seven Bungalows,"Andheri,Western Suburbs",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
10,Clothing Store,2,Bandstand Promenade,"Bandra,Western Suburbs",0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
13,Concert Hall,1,Bandstand Promenade,"Bandra,Western Suburbs",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
14,Department Store,2,Kherwadi,"Bandra,Western Suburbs",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
19,Hotel,1,I.C. Colony,"Borivali (West),Western Suburbs",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
23,Market,1,Gorai,"Borivali (West),Western Suburbs",1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
24,Men's Store,1,Dahisa,Western Suburbs,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
26,Music Venue,1,Thakur village,"Kandivali East,Western Suburbs",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# Cluster the areas

In [114]:
# set number of clusters
kclusters = 5

Mum_clustering = Mum_onehot_new.drop(["col","Area", "Location in Mumbai"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Mum_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 1, 2, 1, 0, 1, 0, 0, 0, 0])

In [115]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
Mum_merged = Mum_onehot_new.copy()

# add clustering labels
Mum_merged["Cluster Labels"] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Mum_merged = Mum_merged.join(Mumbai_df.drop([ "Location in Mumbai"], 1).set_index("Area"), on="Area")

print(Mum_merged.shape)
Mum_merged.head() # check the last columns!


(19, 52)


,col,count,Area,Location in Mumbai,Aquarium,Arcade,Asian Restaurant,Bakery,Bar,Bistro,Burger Joint,Café,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Department Store,Dessert Shop,Fast Food Restaurant,Food Truck,Hookah Bar,Hotel,Ice Cream Shop,Indian Restaurant,Lounge,Market,Men's Store,Middle Eastern Restaurant,Music Venue,Neighborhood,Performing Arts Venue,Platform,Playground,Plaza,Pool,Pub,Restaurant,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shopping Mall,Smoke Shop,Snack Place,South Indian Restaurant,Spanish Restaurant,Train Station,Video Store,Cluster Labels,Lattitude,Longitude
0,Aquarium,1,Amboli,"Andheri,Western Suburbs",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,19.12900,72.84300
1,Arcade,2,"Chakala, Andheri",Western Suburbs,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,19.11138,72.86083
5,Bistro,1,Seven Bungalows,"Andheri,Western Suburbs",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,2,19.12905,72.81701
10,Clothing Store,2,Bandstand Promenade,"Bandra,Western Suburbs",0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,19.04271,72.81913
13,Concert Hall,1,Bandstand Promenade,"Bandra,Western Suburbs",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,19.04271,72.81913


In [116]:
# sort the results by Cluster Labels
print(Mum_merged.shape)
Mum_merged.sort_values(["Cluster Labels"], inplace=True)
Mum_merged

(19, 52)


,col,count,Area,Location in Mumbai,Aquarium,Arcade,Asian Restaurant,Bakery,Bar,Bistro,Burger Joint,Café,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Department Store,Dessert Shop,Fast Food Restaurant,Food Truck,Hookah Bar,Hotel,Ice Cream Shop,Indian Restaurant,Lounge,Market,Men's Store,Middle Eastern Restaurant,Music Venue,Neighborhood,Performing Arts Venue,Platform,Playground,Plaza,Pool,Pub,Restaurant,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shopping Mall,Smoke Shop,Snack Place,South Indian Restaurant,Spanish Restaurant,Train Station,Video Store,Cluster Labels,Lattitude,Longitude
0,Aquarium,1,Amboli,"Andheri,Western Suburbs",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,19.12900,72.84300
36,Scenic Lookout,1,Pali Naka,"Khar,Western Suburbs",0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,19.06274,72.82939
30,Playground,1,Pali Naka,"Khar,Western Suburbs",0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,19.06274,72.82939
28,Performing Arts Venue,1,Thakur village,"Kandivali East,Western Suburbs",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,19.21020,72.87290
27,Neighborhood,1,Thakur village,"Kandivali East,Western Suburbs",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,19.21020,72.87290
24,Men's Store,1,Dahisa,Western Suburbs,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,19.25006,72.85934
23,Market,1,Gorai,"Borivali (West),Western Suburbs",1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,19.25005,72.78202
26,Music Venue,1,Thakur village,"Kandivali East,Western Suburbs",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,19.21020,72.87290
13,Concert Hall,1,Bandstand Promenade,"Bandra,Western Suburbs",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,19.04271,72.81913
19,Hotel,1,I.C. Colony,"Borivali (West),Western Suburbs",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,19.24703,72.84980


In [117]:
# create map
map_clusters = folium.Map(location=[lat, lng], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, area, locat, cluster in zip(Mum_merged['Lattitude'], Mum_merged['Longitude'], Mum_merged['Area'], Mum_merged['Location in Mumbai'],Mum_merged['Cluster Labels']):
    label = folium.Popup(str(area) + ' (Cluster ' + str(cluster + 1) + ')',parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=3,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.2).add_to(map_clusters)
       
map_clusters

# 17. Examine Clusters

In [118]:
Mum_merged.loc[Mum_merged['Cluster Labels'] == 0]

,col,count,Area,Location in Mumbai,Aquarium,Arcade,Asian Restaurant,Bakery,Bar,Bistro,Burger Joint,Café,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Department Store,Dessert Shop,Fast Food Restaurant,Food Truck,Hookah Bar,Hotel,Ice Cream Shop,Indian Restaurant,Lounge,Market,Men's Store,Middle Eastern Restaurant,Music Venue,Neighborhood,Performing Arts Venue,Platform,Playground,Plaza,Pool,Pub,Restaurant,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shopping Mall,Smoke Shop,Snack Place,South Indian Restaurant,Spanish Restaurant,Train Station,Video Store,Cluster Labels,Lattitude,Longitude
0,Aquarium,1,Amboli,"Andheri,Western Suburbs",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,19.12900,72.84300
36,Scenic Lookout,1,Pali Naka,"Khar,Western Suburbs",0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,19.06274,72.82939
30,Playground,1,Pali Naka,"Khar,Western Suburbs",0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,19.06274,72.82939
28,Performing Arts Venue,1,Thakur village,"Kandivali East,Western Suburbs",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,19.21020,72.87290
27,Neighborhood,1,Thakur village,"Kandivali East,Western Suburbs",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,19.21020,72.87290
24,Men's Store,1,Dahisa,Western Suburbs,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,19.25006,72.85934
23,Market,1,Gorai,"Borivali (West),Western Suburbs",1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,19.25005,72.78202
26,Music Venue,1,Thakur village,"Kandivali East,Western Suburbs",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,19.21020,72.87290
13,Concert Hall,1,Bandstand Promenade,"Bandra,Western Suburbs",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,19.04271,72.81913
19,Hotel,1,I.C. Colony,"Borivali (West),Western Suburbs",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,19.24703,72.84980


In [119]:
Mum_merged.loc[Mum_merged['Cluster Labels'] == 1]

,col,count,Area,Location in Mumbai,Aquarium,Arcade,Asian Restaurant,Bakery,Bar,Bistro,Burger Joint,Café,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Department Store,Dessert Shop,Fast Food Restaurant,Food Truck,Hookah Bar,Hotel,Ice Cream Shop,Indian Restaurant,Lounge,Market,Men's Store,Middle Eastern Restaurant,Music Venue,Neighborhood,Performing Arts Venue,Platform,Playground,Plaza,Pool,Pub,Restaurant,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shopping Mall,Smoke Shop,Snack Place,South Indian Restaurant,Spanish Restaurant,Train Station,Video Store,Cluster Labels,Lattitude,Longitude
14,Department Store,2,Kherwadi,"Bandra,Western Suburbs",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,19.05500,72.83100
10,Clothing Store,2,Bandstand Promenade,"Bandra,Western Suburbs",0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,19.04271,72.81913
1,Arcade,2,"Chakala, Andheri",Western Suburbs,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,19.11138,72.86083


In [120]:
Mum_merged.loc[Mum_merged['Cluster Labels'] == 2]

,col,count,Area,Location in Mumbai,Aquarium,Arcade,Asian Restaurant,Bakery,Bar,Bistro,Burger Joint,Café,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Department Store,Dessert Shop,Fast Food Restaurant,Food Truck,Hookah Bar,Hotel,Ice Cream Shop,Indian Restaurant,Lounge,Market,Men's Store,Middle Eastern Restaurant,Music Venue,Neighborhood,Performing Arts Venue,Platform,Playground,Plaza,Pool,Pub,Restaurant,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shopping Mall,Smoke Shop,Snack Place,South Indian Restaurant,Spanish Restaurant,Train Station,Video Store,Cluster Labels,Lattitude,Longitude
5,Bistro,1,Seven Bungalows,"Andheri,Western Suburbs",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,2,19.12905,72.81701
32,Pool,1,Pali Naka,"Khar,Western Suburbs",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,2,19.06274,72.82939


In [121]:
Mum_merged.loc[Mum_merged['Cluster Labels'] == 3]

,col,count,Area,Location in Mumbai,Aquarium,Arcade,Asian Restaurant,Bakery,Bar,Bistro,Burger Joint,Café,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Department Store,Dessert Shop,Fast Food Restaurant,Food Truck,Hookah Bar,Hotel,Ice Cream Shop,Indian Restaurant,Lounge,Market,Men's Store,Middle Eastern Restaurant,Music Venue,Neighborhood,Performing Arts Venue,Platform,Playground,Plaza,Pool,Pub,Restaurant,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shopping Mall,Smoke Shop,Snack Place,South Indian Restaurant,Spanish Restaurant,Train Station,Video Store,Cluster Labels,Lattitude,Longitude
29,Platform,2,Pali Naka,"Khar,Western Suburbs",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,19.06274,72.82939
31,Plaza,2,Pali Naka,"Khar,Western Suburbs",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,19.06274,72.82939


In [122]:
Mum_merged.loc[Mum_merged['Cluster Labels'] == 4]

,col,count,Area,Location in Mumbai,Aquarium,Arcade,Asian Restaurant,Bakery,Bar,Bistro,Burger Joint,Café,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Department Store,Dessert Shop,Fast Food Restaurant,Food Truck,Hookah Bar,Hotel,Ice Cream Shop,Indian Restaurant,Lounge,Market,Men's Store,Middle Eastern Restaurant,Music Venue,Neighborhood,Performing Arts Venue,Platform,Playground,Plaza,Pool,Pub,Restaurant,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shopping Mall,Smoke Shop,Snack Place,South Indian Restaurant,Spanish Restaurant,Train Station,Video Store,Cluster Labels,Lattitude,Longitude
43,Train Station,3,Khar Danda,"Khar,Western Suburbs",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,19.06859,72.84004


# Observations

Most of the Locations where food place is not a most visited placed are in cluster 0 which is around Khar and Borlivali/Kandivali area and also all these places are generally where people like to visit.
This these will be best location for opening a fast food resturant to have good profit for the business
